In [0]:
%sql  SELECT DISTINCT CNTRY FROM workspace.bronze.erp_loc_a101

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DataType, IntegerType
from pyspark.sql.functions import col, trim, length
from pyspark.sql.window import Window

# Read bronze table

In [0]:
df = spark.table("workspace.bronze.erp_loc_a101")

# Silver Transformation

## Trim

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

## Clean customer ID

In [0]:
df = df.withColumn("CID", F.regexp_replace(col("CID"), "-", ""))

## Clean country

In [0]:
df = df.withColumn(
    "cntry",
    F.when(col("cntry").isin("US", "USA"), "United States")
    .when(col("cntry") == "DE", "Germany")
    .when((col("cntry") == "") | (col("cntry").isNull()), "n/a")
    .otherwise(col("cntry"))
)

## Rename columns

In [0]:
RENAME_MAP = {
    "CID": "customer_id",
    "CNTRY": "country"
}

for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

## Sanity check

In [0]:
# df.display()

## Write to silver layer

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")

## Sanity check

In [0]:
%sql
SELECT * FROM workspace.silver.erp_customer_location